<a href="https://colab.research.google.com/github/aayush1607/deepfake_detection/blob/main/Machine_Learning_based_solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

  ### **PIXELS + PCA + LDA CLASSIFICATION**

### 1. Import Necessary Libraries

In [16]:
import dlib
import cv2
import os
import json
from pylab import *
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from tensorflow.keras.models import load_model
import pickle

Connect Gdrive

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


### 2. Frame Extraction from videos

In [ ]:
train_frame_folder = '/content/drive/MyDrive/dataset'

#loading the data from json file
with open(os.path.join('/content/drive/MyDrive/dataset/metadata.json'), 'r') as file:
    data = json.load(file)
    
# Saving the names of videos in a list
list_of_train_data = []

for f in os.listdir(train_frame_folder):
    if f.endswith('.mp4'):
        list_of_train_data.append(f)

detector = dlib.get_frontal_face_detector()
total_frames, total_fake_frames, total_real_frames = 0,0,0

for vid in list_of_train_data:
    count = 0
    cap = cv2.VideoCapture(os.path.join(train_frame_folder, vid))
    frameRate = cap.get(5)
    while cap.isOpened():
        frameId = cap.get(1)
        ret, frame = cap.read()
        if ret != True:
            break
        if frameId % ((int(frameRate)+1)*1) == 0:
            face_rects, scores, idx = detector.run(frame, 0)
            for i, d in enumerate(face_rects):
                x1 = d.left()
                y1 = d.top()
                x2 = d.right()
                y2 = d.bottom()
                crop_img = frame[y1:y2, x1:x2]
                # Resizing the cropped image and saving it to dataset folder in png format
                if data[vid]['label'] == 'REAL':
                  try:
                    cv2.imwrite('dataset/real/'+vid.split('.')[0]+'_'+str(count)+'.png', cv2.resize(crop_img, (224, 224)))
                    count+=1
                    total_real_frames+=1
                  except:
                    pass
                elif data[vid]['label'] == 'FAKE':
                  try:
                    cv2.imwrite('dataset/fake/'+vid.split('.')[0]+'_'+str(count)+'.png', cv2.resize(crop_img, (224, 224)))
                    count+=1
                    total_fake_frames+=1
                  except:
                    pass
    print('Frames for video - '+str(vid) +" : "+str(count))
    total_frames+=count

print("Total Videos - "+str(len(list_of_train_data)))
print("Total Fake Frames Extracted - "+str(total_fake_frames))
print("Total Real Frames Extracted - "+str(total_real_frames))
print("Total Frames Extracted - "+str(total_frames))

!zip -r /content/dataset.zip /content/dataset
from google.colab import files
files.download("/content/dataset.zip")

Frames for video - dtjcyzgdts.mp4 : 7
Frames for video - fckxaqjbxk.mp4 : 7
Frames for video - ihnhxathkq.mp4 : 2
Frames for video - fzvpbrzssi.mp4 : 4
Frames for video - htorvhbcae.mp4 : 6
Frames for video - auquqlmsts.mp4 : 0
Frames for video - fedcdqtboz.mp4 : 8
Frames for video - hqzwudvhih.mp4 : 10
Frames for video - hjxckindzx.mp4 : 4
Frames for video - bqvhlqlrba.mp4 : 7
Frames for video - eurhttghny.mp4 : 5
Frames for video - gbgtzspdnv.mp4 : 4
Frames for video - epeyujjlhn.mp4 : 0
Frames for video - gqnkarjtmh.mp4 : 4
Frames for video - euzlhrwolz.mp4 : 4
Frames for video - epwesrdmpt.mp4 : 1
Frames for video - eisofhptvk.mp4 : 3
Frames for video - btizoxwqwp.mp4 : 8
Frames for video - gvlraqlweh.mp4 : 10
Frames for video - hgjdosshpt.mp4 : 5
Frames for video - dhjnjkzuhq.mp4 : 10
Frames for video - fswszwicfw.mp4 : 1
Frames for video - ejhhdlzpjg.mp4 : 4
Frames for video - espkiocpxq.mp4 : 10
Frames for video - gxzdueambi.mp4 : 1
Frames for video - cxgieqcozm.mp4 : 1
Frames f

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [6]:
!unzip "/content/drive/dataset.zip" -d "/"

Archive:  /content/drive/MyDrive/dataset.zip
   creating: /content/content/dataset/
   creating: /content/content/dataset/real/
  inflating: /content/content/dataset/real/chqqxfuuzi_8.png  
  inflating: /content/content/dataset/real/hivnldfvyl_2.png  
  inflating: /content/content/dataset/real/eppyqpgewp_5.png  
  inflating: /content/content/dataset/real/gnmmhlbzge_0.png  
  inflating: /content/content/dataset/real/eiwtggvtfp_7.png  
  inflating: /content/content/dataset/real/fdpisghkmd_0.png  
  inflating: /content/content/dataset/real/aayrffkzxn_1.png  
  inflating: /content/content/dataset/real/fzvpbrzssi_1.png  
  inflating: /content/content/dataset/real/apedduehoy_3.png  
  inflating: /content/content/dataset/real/gcdtglsoqj_7.png  
  inflating: /content/content/dataset/real/hivnldfvyl_4.png  
  inflating: /content/content/dataset/real/chqqxfuuzi_5.png  
  inflating: /content/content/dataset/real/fgobmbcami_1.png  
  inflating: /content/content/dataset/real/fufcmupzen_8.png  
  in

### 3. Preprocessing for fitting to Classification model

In [9]:

data_dir = 'dataset'


#Storing the real and fake image names in lists
real_data = []
fake_data = []

for f in os.listdir(data_dir+'/real'):
    if f.endswith('.png'):
        real_data.append(f)
    
for f in os.listdir(data_dir+'/fake'):
    if f.endswith('.png'):
        fake_data.append(f)    

#Creating an empty array to store the real and fake image data
X = []

#Creating an empty array store the labels
Y = []


#Appending Numpy array info of image to X and label info to Y (1 for real and 0 for fake)
for img in real_data:
    X.append(img_to_array(load_img(data_dir+'/real/'+img)).flatten() / 255.0)
    Y.append(1)
for img in fake_data:
    X.append(img_to_array(load_img(data_dir+'/fake/'+img)).flatten() / 255.0)
    Y.append(0)

#Normalization
X = np.array(X)
Y = np.array(Y)


#Reshape
# X = X.reshape(-1, 224, 224, 3)
# print(X)
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size = 0.25, random_state=5)
X_train_org = X_train.copy()
X_val_org = X_val.copy()
Y_train_org = Y_train.copy()
Y_val_org = Y_val.copy()

print(X_train)
print(X_val)
print()
print('X_train.shape ',X_train.shape)
print('X_val.shape ', X_val.shape)
print('Y_train.shape ', Y_train.shape)
print('Y_val.shape', Y_val.shape)

data= {}
data["X_train"] = X_train
data["X_val"] = X_val
data["Y_train"] = Y_train
data["Y_val"] = Y_val

output = open('train_1000.pkl', 'wb')
pickle.dump(data, output)
output.close()

!zip -r /content/training_dataset.zip /content/train_1000.pkl
from google.colab import files
files.download("/content/training_dataset.zip")

[[0.30588236 0.21176471 0.1254902  ... 0.43137255 0.3137255  0.25490198]
 [0.29411766 0.21960784 0.16470589 ... 0.40392157 0.29803923 0.23529412]
 [0.49411765 0.3882353  0.36862746 ... 0.6509804  0.59607846 0.41960785]
 ...
 [0.15294118 0.05098039 0.08627451 ... 0.5647059  0.50980395 0.45490196]
 [0.08627451 0.00392157 0.0627451  ... 0.37254903 0.3019608  0.21960784]
 [0.78039217 0.5372549  0.42352942 ... 0.6627451  0.6156863  0.5686275 ]]
[[0.47058824 0.07058824 0.1882353  ... 0.44313726 0.39607844 0.35686275]
 [0.12156863 0.10588235 0.14901961 ... 0.84705883 0.61960787 0.5176471 ]
 [0.14901961 0.1254902  0.09803922 ... 0.6392157  0.5686275  0.49411765]
 ...
 [0.6862745  0.6        0.5686275  ... 0.50980395 0.4392157  0.42745098]
 [0.18039216 0.10588235 0.12156863 ... 0.8117647  0.8509804  0.8745098 ]
 [0.32156864 0.2627451  0.19215687 ... 0.4        0.34509805 0.23529412]]

X_train.shape  (1882, 150528)
X_val.shape  (628, 150528)
Y_train.shape  (1882,)
Y_val.shape (628,)
  adding: co

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [3]:

pkl_file = open('train_1000.pkl', 'rb')
data = pickle.load(pkl_file)
pkl_file.close()
X_train, X_train_org = data["X_train"], data["X_train"]
X_val, X_val_org = data["X_val"], data["X_val"]
Y_train, Y_train_org = data["Y_train"], data["Y_train"]
Y_val, Y_val_org = data["Y_val"], data["Y_val"]


### 4. PCA 

In [4]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 0.99)
X_train = pca.fit_transform(X_train)
X_val = pca.transform(X_val)

In [5]:
data= {}
data["X_train"] = X_train
data["X_val"] = X_val
data["Y_train"] = Y_train
data["Y_val"] = Y_val

output = open('train_pca.pkl', 'wb')
pickle.dump(data, output)
output.close()

!zip -r /content/train_pca.zip /content/train_pca.pkl
from google.colab import files
files.download("/content/train_pca.zip")

updating: content/train_pca.pkl (deflated 7%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [6]:
print(X_train)
print(X_val)
print()
print('X_train.shape ',X_train.shape)
print('X_val.shape ', X_val.shape)
print('Y_train.shape ', Y_train.shape)
print('Y_val.shape', Y_val.shape)

[[ 6.0941057e+00 -3.9693539e+01 -1.1994439e+01 ...  5.6649745e-01
  -5.2161396e-01  8.2733446e-01]
 [ 2.4977500e+00 -4.1770912e+01 -8.4096794e+00 ... -5.2774935e-03
   6.4413272e-02  1.6620064e-01]
 [-1.5972754e+01  2.7640114e+00  2.4767685e+01 ... -4.8748191e-02
  -8.1869252e-02 -4.1491400e-02]
 ...
 [-3.9722080e+01  2.3562513e+01 -2.9240164e+01 ... -1.5184137e-02
   4.4514009e-01 -1.3858339e-01]
 [-7.3696533e+01  1.2498075e+01 -1.0253724e+01 ... -1.8444099e-03
  -7.7670194e-02  2.2236118e-01]
 [ 5.8305214e+01  3.2079987e+01 -3.6038215e+00 ... -5.5225268e-02
   1.6385378e-01 -2.5242740e-01]]
[[ 5.61956482e+01  2.42000446e+01 -2.42797623e+01 ... -4.59584117e-01
   8.60179424e-01  1.81568891e-01]
 [ 2.05072117e+01  1.79289951e+01  2.03549461e+01 ... -2.94669330e-01
   6.07042611e-01  3.42890888e-01]
 [-2.15453281e+01  1.91467381e+01  5.10957098e+00 ... -3.56993228e-01
   6.37999550e-02 -7.78667092e-01]
 ...
 [ 7.78311205e+00 -1.10404730e+01  2.28954124e+01 ...  3.22104841e-01
  -1.06739

### 5. LR Classification

In [17]:
pkl_file = open('train_pca.pkl', 'rb')
data = pickle.load(pkl_file)
pkl_file.close()
X_train = data["X_train"]
X_val = data["X_val"]
Y_train = data["Y_train"]
Y_val = data["Y_val"]

In [7]:

from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(max_iter=1000)
classifier.fit(X_train, Y_train)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [9]:
from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = classifier.predict(X_val)
print(len(y_pred))
print(len(Y_val))
cm = confusion_matrix(Y_val, y_pred)
print(cm)
print('accuracy ->', 100*accuracy_score(Y_val, y_pred))

628
628
[[545  23]
 [ 44  16]]
accuracy -> 89.3312101910828


### 6. LDA 

In [10]:
print(X_train)
print(X_val)
print()
print('X_train.shape ',X_train.shape)
print('X_val.shape ', X_val.shape)
print('Y_train.shape ', Y_train.shape)
print('Y_val.shape', Y_val.shape)

[[ 6.0941057e+00 -3.9693539e+01 -1.1994439e+01 ...  5.6649745e-01
  -5.2161396e-01  8.2733446e-01]
 [ 2.4977500e+00 -4.1770912e+01 -8.4096794e+00 ... -5.2774935e-03
   6.4413272e-02  1.6620064e-01]
 [-1.5972754e+01  2.7640114e+00  2.4767685e+01 ... -4.8748191e-02
  -8.1869252e-02 -4.1491400e-02]
 ...
 [-3.9722080e+01  2.3562513e+01 -2.9240164e+01 ... -1.5184137e-02
   4.4514009e-01 -1.3858339e-01]
 [-7.3696533e+01  1.2498075e+01 -1.0253724e+01 ... -1.8444099e-03
  -7.7670194e-02  2.2236118e-01]
 [ 5.8305214e+01  3.2079987e+01 -3.6038215e+00 ... -5.5225268e-02
   1.6385378e-01 -2.5242740e-01]]
[[ 5.61956482e+01  2.42000446e+01 -2.42797623e+01 ... -4.59584117e-01
   8.60179424e-01  1.81568891e-01]
 [ 2.05072117e+01  1.79289951e+01  2.03549461e+01 ... -2.94669330e-01
   6.07042611e-01  3.42890888e-01]
 [-2.15453281e+01  1.91467381e+01  5.10957098e+00 ... -3.56993228e-01
   6.37999550e-02 -7.78667092e-01]
 ...
 [ 7.78311205e+00 -1.10404730e+01  2.28954124e+01 ...  3.22104841e-01
  -1.06739

In [11]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA


lda = LDA()
lda.fit(X_train, Y_train)
y_prob_lda = lda.predict_proba(X_val)[:,1]
y_pred_lda = np.where(y_prob_lda > 0.5, 1, 0)

print(y_pred_lda)

print(X_train)
print(X_val)
print()
print('X_train.shape ',X_train.shape)
print('X_val.shape ', X_val.shape)
print('Y_train.shape ', Y_train.shape)
print('Y_val.shape', Y_val.shape)

[0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0
 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0
 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 1 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0
 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0
 0 0 0 0 0 0 0 0 0 0 0 0 

In [12]:
lda_confusion_matrix = confusion_matrix(Y_val, y_pred_lda)
print(lda_confusion_matrix)


[[541  27]
 [ 45  15]]


In [14]:
from sklearn.metrics import roc_curve, auc
false_positive_rate, true_positive_rate, thresholds = roc_curve(Y_val, y_prob_lda)
roc_auc_lda = auc(false_positive_rate, true_positive_rate)
print(100*roc_auc_lda)

76.9043427230047
